In [2]:
import btrack
import zarr
import os
import napari
from macrohet import dataio
import glob
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
from trackastra.model import Trackastra
from trackastra.tracking import graph_to_ctc, graph_to_napari_tracks

!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics
use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')
cellpose_model = models.Cellpose(gpu=True, model_type='cyto')

device = "cuda" if torch.cuda.is_available() else "cpu"
# Load a pretrained model
trackastra_model = Trackastra.from_pretrained("general_2d", device=device)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:17:15_PST_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0
Fri Oct 25 11:04:39 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:65:00.0  On |                  Off |
| 30%   43C    P8      

INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU (CUDA)
INFO:cellpose.models:>> cyto << model set to be used


>>> GPU activated? YES


INFO:cellpose.models:>>>> loading model /home/dayn/.cellpose/models/cytotorch_0
INFO:cellpose.models:>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


/home/dayn/.trackastra/.models/general_2d already downloaded, skipping.


INFO:trackastra.model.model:Loading model state from /home/dayn/.trackastra/.models/general_2d/model.pt
INFO:trackastra.model.model_api:Using device cuda


In [2]:
expt_ID = 'ND0004'
base_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/'
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,replicate_number=True)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
acq_ID = (3, 4)
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
zarr_group = zarr.open(image_dir, mode='r')

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


In [3]:
%%time
images = zarr_group.images[...]

CPU times: user 45.1 s, sys: 35.9 s, total: 1min 20s
Wall time: 5min 49s


In [4]:
with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/cpv3/{acq_ID}.h5'), #macrohet_seg_model 
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                segmentation = reader.segmentation

[INFO][2024/10/24 02:54:45 PM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(3, 4).h5...
[INFO][2024/10/24 02:55:06 PM] Loading segmentation (154, 6048, 6048)
[INFO][2024/10/24 02:55:06 PM] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(3, 4).h5


In [5]:
tracks = pd.read_csv('/mnt/SYNO/macrohet_syno/data/ND0004/labels/testing_trackastra/ztracks.csv')

In [6]:
tracks = tracks[tracks.groupby('ID')['T'].transform('count') >= 130]


In [11]:
%%time
images = np.max(images, axis = 2)

CPU times: user 14 s, sys: 29 s, total: 43 s
Wall time: 43.2 s


In [12]:
images.shape

(154, 2, 6048, 6048)

In [13]:
viewer = napari.Viewer(title = 'testing nd4 segmentation and tracking')

viewer.add_image(images, channel_axis = 1, 
                 colormap=['magenta', 'green'],
                 blending = 'additive', 
                 contrast_limits=[[0, 1000], [0, 2400]])
viewer.add_labels(segmentation)

<Labels layer 'segmentation' at 0x7f3e049b6e90>

In [13]:
scale = (1,1,1)

In [15]:
color_by = 'ID'

In [62]:
# Initialize relabeled array
relabeled = np.zeros_like(segmentation)

# Iterate over each track, grouped by 'ID'
for cell_ID, track in tqdm(tracks.groupby('ID'), desc = 'Iterating over tracks', total = len(tracks.ID.unique())):
    track = track.sort_values(by='T')
    times = (track['T'].to_numpy()).astype(int)  # Time (T) halved and converted to int
    x_coords = (track['X'].to_numpy() * scale[0]).astype(int)
    y_coords = (track['Y'].to_numpy() * scale[1]).astype(int)

    # Iterate over each time point
    for i, t in enumerate(times):
        # Ensure we are within the segmentation time bounds
        if t >= segmentation.shape[0]:
            continue

        # Handle 2D segmentation
        old_id = segmentation[t][x_coords[i], y_coords[i]]
        if old_id == 0:  # Ignore background (assuming 0 is the background ID)
                continue
        # Recolor segmentation by the chosen property (ID in this case)
        old_id_mask = segmentation[t] == old_id
        # print(old_id)
        # Recolor all pixels in this mask with the new ID
        new_id = int(cell_ID)
        relabeled[t][old_id_mask] = new_id

Iterating over tracks:   0%|          | 0/162 [00:00<?, ?it/s]

In [63]:
viewer.add_labels(relabeled)

<Labels layer 'relabeled [1]' at 0x7fc7e60dce80>

In [14]:
tracks

,Unnamed: 0,ID,T,X,Y
245,245,8.0,0.0,1316.925443,1308.419429
246,246,8.0,1.0,1296.484605,1236.577458
247,247,8.0,2.0,1301.557163,1220.274875
248,248,8.0,3.0,1290.174829,1210.652110
249,249,8.0,4.0,1330.836471,1218.764602
...,...,...,...,...,...
61876,61876,2283.0,149.0,4957.993444,2788.832197
61877,61877,2283.0,150.0,4961.470775,2788.063490
61878,61878,2283.0,151.0,4957.135691,2776.889461
61879,61879,2283.0,152.0,4963.660649,2778.348608


In [15]:
viewer.add_tracks(tracks[['ID','T','X','Y']].values)

<Tracks layer 'Tracks' at 0x7f3dc435d660>

# Testing new approach

Segment the top z slice of the green channel... can always expand the masks later on to try and capture the mtb

In [16]:
zarr_group.images.shape

(154, 2, 3, 6048, 6048)

In [23]:
%%time
segmentation_input = zarr_group.images[:,1,2,...]

CPU times: user 9.72 s, sys: 5.24 s, total: 15 s
Wall time: 54.8 s


In [24]:
viewer.add_image(segmentation_input)

<Image layer 'segmentation_input' at 0x7f93e45de170>

In [59]:
mask_stack = []
for frame in tqdm(segmentation_input, total = len(segmentation_input)):
    masks, flows, styles, diams = model.eval(frame, 
                                         diameter=150, 
                                         channels=[0,0],)

    mask_stack.append(masks)
cellpose_segmentation = np.stack(mask_stack, axis = 0)

  0%|          | 0/154 [00:00<?, ?it/s]

In [65]:
# Save the NumPy array to a file
np.save('/mnt/SYNO/macrohet_syno/temp_cellpose_masks.npy', cellpose_segmentation)

In [96]:
viewer.add_labels(cellpose_segmentation)

<Labels layer 'cellpose_segmentation' at 0x7f951d8f6fe0>

In [67]:
viewer.add_image(segmentation_input)

<Image layer 'segmentation_input' at 0x7f957b65df90>

# Trackastra

In [ ]:
%%time 

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load a pretrained model
trackastra_model = Trackastra.from_pretrained("general_2d", device=device)

# or from a local folder
# model = Trackastra.from_folder('path/my_model_folder/', device=device)

# Track the cells
track_graph = trackastra_model.track(segmentation_input, cellpose_segmentation, mode="greedy_nodiv")  # or mode="ilp", or "greedy_nodiv"


# # Write to cell tracking challenge format
# ctc_tracks, masks_tracked = graph_to_ctc(
#       track_graph,
#       masks,
#       outdir="tracked",
# )

# Visualise in napari
tracks, napari_tracks_graph, _ = graph_to_napari_tracks(track_graph)

In [74]:
tracks = viewer.layers['tracks'].data

In [78]:
tracks

array([[1.00000000e+00, 0.00000000e+00, 5.63499262e+03, 5.98160372e+03],
       [1.00000000e+00, 1.00000000e+00, 5.62536602e+03, 5.96917297e+03],
       [1.00000000e+00, 2.00000000e+00, 5.61263586e+03, 5.95262619e+03],
       ...,
       [6.97800000e+03, 1.53000000e+02, 5.03416957e+03, 5.25425019e+03],
       [6.97900000e+03, 1.52000000e+02, 5.29644164e+03, 6.23781633e+02],
       [6.97900000e+03, 1.53000000e+02, 5.32309206e+03, 6.78151989e+02]])

# uniting z3 cellpose and trackastra

In [3]:
# define thresholds
segment_size_thresh = 5000
mtb_load_thresh = 480
mtb_channel = 0

In [4]:
mtb_channel_ID = 0
mphi_channel_ID = 1

In [24]:
# mphi_channel = images[:,mphi_channel,...]
mtb_channel = images[:,mtb_channel,...]
thresholded_mtb_channel = mtb_channel >= mtb_load_thresh

In [25]:
viewer.add_image(thresholded_mtb_channel)

<Image layer 'thresholded_mtb_channel' at 0x7f3e063d31f0>

In [ ]:
with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/cpv3/{acq_ID}.h5'), #macrohet_seg_model 
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                segmentation = reader.segmentation

In [5]:
def calculate_msd(x, y):
    # Calculate the displacement between successive frames
    dx = np.diff(x, prepend=x[0])
    dy = np.diff(y, prepend=y[0])
    msd = np.sqrt(dx**2 + dy**2)
    return msd

In [6]:
# Assuming you have an 'image' that corresponds to your segmentation
def measure_segment_intensity(image, segmentation, ID):
    """Measure intensity of pixels under a specific segment in the image."""
    mask = segmentation == ID  # Create a mask for the segment with 'seg_id'
    segment_intensity = image[mask]  # Extract pixel values under the segment mask
    return np.mean(segment_intensity) #, np.sum(segment_intensity)  # Example measurements


In [110]:
mtb_channel = 0
mphi_channel = 1

In [70]:
del tracks['Unnamed: 0']

In [72]:
tracks.rename(columns={'T': 't', 'X': 'x', 'Y': 'y'}, inplace=True)

In [74]:
tracks = tracks.astype(int)

tracks

,ID,t,x,y
245,8,0,1316,1308
246,8,1,1296,1236
247,8,2,1301,1220
248,8,3,1290,1210
249,8,4,1330,1218
...,...,...,...,...
61876,2283,149,4957,2788
61877,2283,150,4961,2788
61878,2283,151,4957,2776
61879,2283,152,4963,2778


In [108]:
images.shape

(154, 2, 6048, 6048)

In [133]:
#technical replicate
technical_replicate = assay_layout.loc[(acq_ID[0], acq_ID[1])]['Replicate #']
#biological replicate
biological_replicate = 4
#strain
strain = assay_layout.loc[(acq_ID[0], acq_ID[1])]['Strain']
#compound
compound = assay_layout.loc[(acq_ID[0], acq_ID[1])]['Strain']
#concentration
concentration = assay_layout.loc[(acq_ID[0], acq_ID[1])]['Concentration']

# load the original cellpose segmentation
segmentation = segmentation
# calculate the new cellpose segmentation
tracking_input_segmentation = None
# track using new segmentation
tracks = tracks
# load images and max project them
images = images
# split channels
mphi_channel = images[:,mphi_channel_ID,...]
mtb_channel = images[:,mtb_channel_ID,...]
thresholded_mtb_channel = mtb_channel >= Mtb_load_thresh

track_dfs = []
# now measure properties from prior segmentation?
for cell_ID, track in tqdm(tracks.groupby('ID'), desc = 'iterating over tracks', total = len(tracks.ID.unique())):
    track = track.sort_values(by='t')
    # Extract coordinates and time
    times = track['t'].to_numpy() / 2  # Assuming you want to halve the time
    frames = track['t'].to_numpy() 

    x_coords = track['x'].to_numpy().astype(int)
    y_coords = track['y'].to_numpy().astype(int)


    mtb_areas = []
    mphi_areas = []
    mean_intensities = []
    # calculate the mtb pixel area and µm area
    for i, frame in tqdm(enumerate(frames), desc = f'Iterating over frames for track ID {cell_ID}', total = len(frames), leave = False):
        frame = frame - 1
        segmentation_input_ID = segmentation[frame][x_coords[i], y_coords[i]]
        if segmentation_input_ID == 0:  # Ignore background (assuming 0 is the background ID)
                mtb_area_pixels = np.nan
                mphi_area_pixels = np.nan
        else:
            
            mask = segmentation[frame] == segmentation_input_ID  # Create a mask for the segment with 'seg_id'
            
            # chop up images into segments here
            # image_segment = images[frame][:][mask]
            thresholded_image_segment = thresholded_mtb_channel[frame][mask]
    
            # meaure segment
            mtb_area_pixels = np.sum(thresholded_image_segment)
            mphi_area_pixels = np.sum(mask)
            # mean_intensity = np.mean(image_segment)

        # store measurements
        mtb_areas.append(mtb_area_pixels)
        mphi_areas.append(mphi_area_pixels)
        # mean_intensities.append(mean_intensity)

    track['Mtb Area (µm)'] = np.array(mtb_areas) * pixel_to_mum_sq_scale_factor
    track['Mphi Area (µm)'] = np.array(mphi_areas) * pixel_to_mum_sq_scale_factor
    # track['RFP'] = mean_intensities[mtb_channel_ID]
    # track['GFP'] = mean_intensities[mphi_channel_ID]
    # Compute MSD in a vectorized way
    # track['MSD'] = calculate_msd(x_coords, y_coords)
    
    # infection statuses
    track['Infection Status'] = track['Mtb Area (µm)'] > 0

    track['Initial Infection Status'] = track['Mtb Area (µm)'].iloc[0] > 0 
    track['Final Infection Status'] = track['Mtb Area (µm)'].iloc[-1] > 0 

    track['ID'] = f'{cell_ID}.{acq_ID[0]}.{acq_ID[1]}.{expt_ID}'
    
    track_dfs.append(track)

# Concatenate all track DataFrames into the larger 'df' DataFrame
df = pd.concat(track_dfs, ignore_index=True)

df.to_pickle(f'sc_df_{acq_ID[0]}.{acq_ID[1]}.{expt_ID}.pkl')

iterating over tracks:   0%|          | 0/162 [00:00<?, ?it/s]

Iterating over frames for track ID 8:   0%|          | 0/154 [00:00<?, ?it/s]

Iterating over frames for track ID 20:   0%|          | 0/154 [00:00<?, ?it/s]

Iterating over frames for track ID 26:   0%|          | 0/154 [00:00<?, ?it/s]

Iterating over frames for track ID 34:   0%|          | 0/154 [00:00<?, ?it/s]

...

Iterating over frames for track ID 2063:   0%|          | 0/134 [00:00<?, ?it/s]

Iterating over frames for track ID 2174:   0%|          | 0/132 [00:00<?, ?it/s]

Iterating over frames for track ID 2244:   0%|          | 0/131 [00:00<?, ?it/s]

Iterating over frames for track ID 2259:   0%|          | 0/131 [00:00<?, ?it/s]

Iterating over frames for track ID 2283:   0%|          | 0/130 [00:00<?, ?it/s]

In [132]:
x_coords[151]

110

In [129]:
frame

152

In [128]:
frames

array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153])

In [10]:
expt_ID = 'ND0004'
base_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/'
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,replicate_number=True)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
image_resolution = float(metadata['ImageResolutionX'].iloc[0])
meters_area_per_pixel = image_resolution**2
mum_sq_scale_factor = (1E-6)**2
pixel_to_mum_sq_scale_factor = meters_area_per_pixel/mum_sq_scale_factor


# Sort by custom order of the Compound and ConcentrationEC
compound_order = ['RIF', 'PZA', 'INH', 'CTRL', 'BDQ']
concentration_order = ['EC99', 'EC50', 'EC0']

# Define custom sort logic for the DataFrame
assay_layout['compound_sort'] = assay_layout['Compound'].apply(lambda x: compound_order.index(x) if x in compound_order else len(compound_order))
assay_layout['concentration_sort'] = assay_layout['ConcentrationEC'].apply(lambda x: concentration_order.index(x) if x in concentration_order else len(concentration_order))
assay_layout['strain_sort'] = assay_layout['Strain'].apply(lambda x: 0 if x == 'WT' else (1 if x == 'RD1' else 2))

# Sort the DataFrame based on the defined sort order
assay_layout_sorted = assay_layout.sort_values(by=['concentration_sort', 'compound_sort', 'strain_sort'])

# Extract the row-column tuples from the sorted DataFrame index
row_col_order = list(assay_layout_sorted.index)


Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


In [ ]:
for acq_ID in tqdm(row_col_order, total = len(row_col_order), desc = 'Iterating over individual wells'):
    
    if os.path.exists(f'sc_df_{acq_ID[0]}.{acq_ID[1]}.{expt_ID}.pkl'):
        print(f'skipping acq ID {acq_ID}')
        continue
    #technical replicate
    technical_replicate = assay_layout.loc[(acq_ID[0], acq_ID[1])]['Replicate #']
    #biological replicate
    biological_replicate = 4
    #strain
    strain = assay_layout.loc[(acq_ID[0], acq_ID[1])]['Strain']
    #compound
    compound = assay_layout.loc[(acq_ID[0], acq_ID[1])]['Strain']
    #concentration
    concentration = assay_layout.loc[(acq_ID[0], acq_ID[1])]['Concentration']
    # load images and max project them
    image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
    zarr_group = zarr.open(image_dir, mode='r')
    images = zarr_group.images[...]
    # load the original cellpose segmentation
    with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/cpv3/{acq_ID}.h5'), #macrohet_seg_model 
                                               'r', 
                                               obj_type='obj_type_1'
                                               ) as reader:
                    segmentation = reader.segmentation
    # calculate the new cellpose segmentation
    segmentation_input = images[:,1,2,...]
    mask_stack = []
    for frame in tqdm(segmentation_input, total = len(segmentation_input)):
        masks, flows, styles, diams = cellpose_model.eval(frame, 
                                                 diameter=150, 
                                                 channels=[0,0],)
    
        mask_stack.append(masks)
    tracking_input_segmentation = np.stack(mask_stack, axis = 0)
    
    # track using new segmentation
    # Track the cells
    track_graph = trackastra_model.track(segmentation_input, cellpose_segmentation, mode="greedy_nodiv")  # or mode="ilp", or "greedy_nodiv"
    
    
    # Visualise in napari
    tracks, napari_tracks_graph, _ = graph_to_napari_tracks(track_graph)
    tracks = pd.DataFrame(tracks, columns=['ID', 't', 'x', 'y']).astype(int)
    # split channels
    mphi_channel = images[:,mphi_channel_ID,...]
    mtb_channel = images[:,mtb_channel_ID,...]
    thresholded_mtb_channel = mtb_channel >= Mtb_load_thresh
    # now measure properties from prior segmentation?
    for cell_ID, track in tqdm(tracks.groupby('ID'), desc = 'iterating over tracks', total = len(tracks.ID.unique()), leave = False):
        track = track.sort_values(by='t')
        # Extract coordinates and time
        times = track['t'].to_numpy() / 2  # Assuming you want to halve the time
        frames = track['t'].to_numpy() 
    
        x_coords = track['x'].to_numpy().astype(int)
        y_coords = track['y'].to_numpy().astype(int)
    
    
        mtb_areas = []
        mphi_areas = []
        mean_intensities = []
        # calculate the mtb pixel area and µm area
        for i, frame in tqdm(enumerate(frames), desc = f'Iterating over frames for track ID {cell_ID}', total = len(frames), leave = False):
            frame = frame - 1
            segmentation_input_ID = segmentation[frame][x_coords[i], y_coords[i]]
            if segmentation_input_ID == 0:  # Ignore background (assuming 0 is the background ID)
                    mtb_area_pixels = np.nan
                    mphi_area_pixels = np.nan
            else:
                
                mask = segmentation[frame] == segmentation_input_ID  # Create a mask for the segment with 'seg_id'
                
                # chop up images into segments here
                # image_segment = images[frame][:][mask]
                thresholded_image_segment = thresholded_mtb_channel[frame][mask]
        
                # meaure segment
                mtb_area_pixels = np.sum(thresholded_image_segment)
                mphi_area_pixels = np.sum(mask)
                # mean_intensity = np.mean(image_segment)
    
            # store measurements
            mtb_areas.append(mtb_area_pixels)
            mphi_areas.append(mphi_area_pixels)
            # mean_intensities.append(mean_intensity)
    
        track['Mtb Area (µm)'] = np.array(mtb_areas) * pixel_to_mum_sq_scale_factor
        track['Mphi Area (µm)'] = np.array(mphi_areas) * pixel_to_mum_sq_scale_factor
        # track['RFP'] = mean_intensities[mtb_channel_ID]
        # track['GFP'] = mean_intensities[mphi_channel_ID]
        # Compute MSD in a vectorized way
        # track['MSD'] = calculate_msd(x_coords, y_coords)
        
        # infection statuses
        track['Infection Status'] = track['Mtb Area (µm)'] > 0
    
        track['Initial Infection Status'] = track['Mtb Area (µm)'].iloc[0] > 0 
        track['Final Infection Status'] = track['Mtb Area (µm)'].iloc[-1] > 0 
    
        track['ID'] = f'{cell_ID}.{acq_ID[0]}.{acq_ID[1]}.{expt_ID}'
        
        track_dfs.append(track)
    
    # Concatenate all track DataFrames into the larger 'df' DataFrame
    df = pd.concat(track_dfs, ignore_index=True)
    
    df.to_pickle(f'sc_df_{acq_ID[0]}.{acq_ID[1]}.{expt_ID}.pkl')

Iterating over individual wells:   0%|          | 0/42 [00:00<?, ?it/s]

[INFO][2024/10/25 11:14:17 AM] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(4, 7).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(4, 7).h5...
[INFO][2024/10/25 11:14:35 AM] Loading segmentation (154, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (154, 6048, 6048)
[INFO][2024/10/25 11:14:35 AM] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(4, 7).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0004/labels/cpv3/(4, 7).h5


  0%|          | 0/154 [00:00<?, ?it/s]

INFO:cellpose.models:channels set to [0, 0]
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 44.99 sec
INFO:cellpose.models:channels set to [0, 0]
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 37.50 sec
INFO:cellpose.models:channels set to [0, 0]
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 41.17 sec
INFO:cellpose.models:channels set to [0, 0]
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 38.75 sec
INFO:cellpose.models:channels set to [0, 0]
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 37.25 sec
INFO:cellpose.models:channels set to [0, 0]
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 36.46 sec
INFO:cellpose.models:channels set to [0, 0]
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 36.76 sec
INFO:cellpose.models:channels set to [0, 0]
INFO:cellpose.mode